# Leetcode - minPathSum algorithm
## Objective : Determine the path that minimizes the sum from top left to bottom right corner of a m x n grid
Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right which minimizes the sum of all numbers along its path.
Note: You can only move either down or right at any point in time.
Example:
Input:
[
  [1,3,1],
  [1,5,1],
  [4,2,1]
]
Output: 7.  Explanation : Because the path 1→3→1→1→1 minimizes the sum.

Option 1 : Iterate over the grid from the first row to the last and first column to last, quadratic compexity O(m * n)
- When at position i=0 and j=0, do nothing.
- When at position i=0 and j!=0 (first row), assign to grid[i, j] the sum of grid[i, j] and grid[i, j-1]
- When at position i!=0 and j=0 (first column), assign to grid[i, j] the sum of grid[i, j] and grid[i-1, j]
- When at position i!=0 and j!=0 (neither first row nor first column), assign to grid[i, j]
  the sum of grid[i, j] and minimum of (grid[i, j-1], grid[i-1, j])
- At the end of these iterations, the result is found at grid[lastrow, lastcolumn]

Option 2 : List all possible paths, exponential complexity! O(2^(m + n))
    
- The number of rows nbrows is equal to the length of the input grid
- The number of columns nbcols is equal to the length of the first row of the input grid
- The path from the top left to bottom right has a number of points : : nb_of_points_in_paths = nbrows + nbcols - 1
- The number of possible paths from the start to the end is equal to (for nbrows>=3 and nbcols>=3) :
    2 times (2 possible moves at each of these points) the number points in the input grid excluding the last column, 
    last row and start point i.e nb_of_possible_full_paths = ((nbrows-1)*(nbcols-1)-1) * 2 

1. Generate the path of length 1 i.e start point only
2. For all path lengths from 2 to nb_of_points_in_paths
    - Generate recursively the paths of length l, using the paths of length l-1 by
      * Adding 1 path with an additional point horizontally to the right and add 1 path with an additional point vertically 
        downward if the last point of the path of length l-1 considered is neither in the last column nor the last row
      * Adding 1 path with an additional point horizontally to the right if the last point of the path of length l-1 
        considered is in the last row
      * Adding 1 path with an additional point vertically downward if the last point of the path of length l-1 considered is 
        in the last column
3. Get the list of paths of full length at the end of the iteration
4. Compare the ((nbrows-1)*(nbcols-1)-1)*2 paths by adding their path integers
5. Get the minimum sum

* Generation of paths : ~ (2 + 2^2 +...+2^lmax-1) = O(2^lmax) = O(2^(m+n)),
  since lmax = nb_of_points_in_paths = nbrows + nbcols - 1 = m+n-1
Paths of length 1 : 0
Paths of length 2 : ~2
Paths of length 3 : ~2*2
.    
Paths of length l : ~2^lmax-1

* Comparison of paths of length n : ~ O((nbrows-1)*(nbcols-1)-1) * 2 * nb_of_points_in_paths = O((m * n)^2) * O(max(m, n)) = O((m * n)^2 * max(m, n))

Overall complexity : O(2^(m + n)) + O((m * n)^2 * max(m, n)) = O(2^(m + n))

In [1]:
# Option 1 : Quadratic complexity
import time
t1q = time.time()
def minPathSum(grid):
    nbrows = len(grid)
    nbcols = len(grid[0])
    
    for i in range(nbrows):
        for j in range(nbcols):
            
            # Start point
            if i==0 and j==0:
                pass

            # First row
            elif i==0 and j!=0:
                grid[i][j] += grid[i][j-1]
                
            # First column
            elif i!=0 and j==0:
                grid[i][j] += grid[i-1][j]
                
            # Neither first row nor first column, check the min of [i-1,j] and [i,j-1]
            else:
                grid[i][j] += min(grid[i][j-1], grid[i-1][j])
                
    # The smallest sum is the value at the bottom right
    return grid[nbrows-1][nbcols-1]

# Test input
input_list = [[7,1,3,5,8,9,9,2,1,9],
              [9,5,9,4,0,4,8,8,9,5],
              [8,2,9,1,3,1,9,7,2,5],
              [6,7,9,8,4,8,3,0,4,0],
              [7,1,3,1,8,8,3,1,2,1],
              [9,5,4,3,5,6,1,3,6,4]]

print(minPathSum(input_list))
t2q = time.time()

43


In [2]:
# Option 2 : Exponential complexity
t1e = time.time()
def generate_possible_paths(grid):
    nbrows = len(grid)
    nbcols = len(grid[0])
    nb_of_points_in_paths = nbrows + nbcols - 1

    # Put the possible paths of length 1 i.e with a single point at position 1
    possible_paths_of_length_i = [[[0, 0]]]
    
    # Generate the list of possible paths for each length from 2 to nb_of_points_in_paths
    for path_len in range(2, nb_of_points_in_paths+1):

        # Paths of length l = 2 to n
        possible_paths_of_length_i_plus = []
        
        # The possible paths of length l
        # Check all points that can come after the previous point in the path
        
        # Check each path of length l-1
        for path in possible_paths_of_length_i:
                       
            #Read the last point in path of length l-1
            last_point = path[-1]
                
            # If neither on last row nor on last column, add two new paths
            if last_point[1] < nbcols-1 and last_point[0] < nbrows-1:
                # Add the path with last point : [i+1, j]
                possible_paths_of_length_i_plus.append(path + [[last_point[0]+1, last_point[1]]])

                #Add the path with last point : [i, j+1]
                possible_paths_of_length_i_plus.append(path + [[last_point[0], last_point[1]+1]])

            # If on last column but not at destination, move downward only
            elif last_point[1] == nbcols-1 and last_point[0] != nbrows-1:
                # Add the path with last point : [i+1, j]
                possible_paths_of_length_i_plus.append(path + [[last_point[0]+1, last_point[1]]])

            # If on last row but not at destination, move right only
            elif last_point[1] != nbcols-1 and last_point[0] == nbrows-1:
               #Add the path with last point : [i, j+1]
                possible_paths_of_length_i_plus.append(path + [[last_point[0], last_point[1]+1]])

            # The end of the last path is the destination => No additional move
            else :
                pass
            
        # After looping on paths of length l-1 and creating paths of length l, 
        # update the list of paths from length l-1 to those of length l, for the next iteration
        possible_paths_of_length_i = possible_paths_of_length_i_plus
        
    # Return the list of full length paths
    return possible_paths_of_length_i

# Test input
input_list = [[7,1,3,5,8,9,9,2,1,9],
              [9,5,9,4,0,4,8,8,9,5],
              [8,2,9,1,3,1,9,7,2,5],
              [6,7,9,8,4,8,3,0,4,0],
              [7,1,3,1,8,8,3,1,2,1],
              [9,5,4,3,5,6,1,3,6,4]]

# The list of possible full length paths
full_length_paths = generate_possible_paths(input_list)

def compare_paths(full_length_paths, input_list): 
    #best_path = []
    best_path_sum = float('Inf')
    
    # Iterate over paths
    for path in full_length_paths:
        
        path_sum = 0
        #Iterate over points in path
        for point in path:
            # Sum of numbers along path
            path_sum += input_list[point[0]][point[1]]
        
        if path_sum < best_path_sum:
            # This path is the best
            # best_path = path
            best_path_sum = path_sum
            
    return best_path_sum

# Return the path with the shortest sum and the path sum    
print(compare_paths(full_length_paths, input_list))

t2e = time.time()

43


In [3]:
print("Time for Option 1: {}ms".format(1000*round(t2q-t1q, 4)))
print("\nTime for Option 2: {}ms".format(1000*round(t2e-t1e, 4)))

Time for Option 1: 1.0ms

Time for Option 2: 7.0ms


Listing all paths should be avoided due to :
- exponential complexity, increasing the dimensions of the input grid would increase the computation time exponentially
- high memory footprint.
However to obtain the best path in option 1, there is need for more processing in order to memorize the best moves
for each position.